# このドキュメントの目的

学習パラメータの自動的な探索方法について整理することです。

調整するパラメータは機械学習やディープラーニングのものを対象としています。

構成としては、自動パラメータの調整ライブラリ　hyperopt　の使用法を中心にしてまとめています。


# 参考にした文献

https://qiita.com/kenchin110100/items/ac3edb480d789481f134

https://qiita.com/nazoking@github/items/f67f92dc60001a43b7dc


# hyperopt とは？

Distributed Asynchronous Hyperparameter Optimization in Python 

2019/3/30 時点で　970commits, 43contributors, 3200Star, 587Fork のライブラリ

[github] https://github.com/hyperopt/hyperopt

[doccumente] http://hyperopt.github.io/hyperopt/

[公式wiki] https://github.com/hyperopt/hyperopt/wiki/FMin


# インストール

> pip install hyperopt

In [1]:
# 最も単純なhyperoptの使い方

def objective(x):
    return x ** 2

from hyperopt import fmin, tpe, hp
# fmin なので関数を最小化する引数を求める。
# fn=最小化する関数
# space=関数で定義した変数と、それの探索範囲
# algo=探索アルゴリズム
# max_evals=探索の繰り返し回数
best = fmin(
    fn=objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100
    )

best


100%|██████████| 100/100 [00:00<00:00, 469.86it/s, best loss: 0.003039930524044733]


{'x': -0.05513556496531738}

In [2]:
# 以下のようにすれば、処理時間も確認することができます。

import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss' : x ** 2,
        'status' : STATUS_OK, # STATUS_OK の時だけlossを考慮する？
        # -- store other results like this --
        'eval_time' : time.time(),
        'other_stuff' : {'type' : None, 'value' : [0,1,2]},
        # -- attachments are handled differently
        'attachments' : {'time_module' : pickle.dumps(time.time)}        
    }


trials = Trials() #計算中の戻り値を追跡可能にする。
from hyperopt import fmin, tpe, hp
best = fmin(
    fn=objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=3,
    trials=trials
    )

from pprint import pprint as print
print('=== best ===')
print(best)
print('')
print('=== trials.trials ===')
print(trials.trials)
print('')
print('=== trials.losses() ===')
print(trials.losses())
print('')
print('=== trials.statuses() ===')
print(trials.statuses())
print('')

100%|██████████| 3/3 [00:00<00:00, 473.79it/s, best loss: 0.6661991494458527]
'=== best ==='
{'x': -0.8162102360579979}
''
'=== trials.trials ==='
[{'book_time': datetime.datetime(2019, 4, 6, 1, 38, 55, 770000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
           'idxs': {'x': [0]},
           'tid': 0,
           'vals': {'x': [-0.8162102360579979]},
           'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2019, 4, 6, 1, 38, 55, 770000),
  'result': {'eval_time': 1554514735.770375,
             'loss': 0.6661991494458527,
             'other_stuff': {'type': None, 'value': [0, 1, 2]},
             'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 0,
  'version': 0},
 {'book_time': datetime.datetime(2019, 4, 6, 1, 38, 55, 771000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
           'idxs': {'x': [1]},
           'tid': 1,
           'vals': {'x': [1.9971252494478442]},
           

In [3]:
# より複雑な探索空間を扱う場合

## define an objective function
def objective(args):
    case, val = args
    if case == 'case 1':
        return val
    else:
        return val ** 2
    
print( objective(['case 2', 3]) )

9


In [4]:
## 複雑な変数空間を定義する
## define a search space of hyperparameter
from hyperopt import hp
space = hp.choice('a',
                 [
                   ('case 1', 1+hp.lognormal('c1',     0,   1)),
                   ('case 2',          hp.uniform('c2', -10, 10))
                 ])

print('もし上記で定義した変数空間を確認したいなら、以下でサンプリング結果を確認することができます')
import hyperopt.pyll.stochastic
for _ in range(10):
    print(hyperopt.pyll.stochastic.sample(space))

print('--- 次に、探索を行います ---')

# minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

print('=== best ===')
print( best )

print('=== space_eval ===')
print( space_eval(space, best) )

'''注釈：
'a', 'c1', 'c2' は任意に指定できるラベルで、fminが内部的な命名に利用しています。
上記の場合、'a'はケース名として機能し、以下のような意味合いを持ちます。
if a == 0:
    args = ('case 1', 1+hp.lognormal('c1',     0,   1))
elif a == 1:
    args = ('case 2',          hp.uniform('c2', -10, 10))
'''
print('')



'もし上記で定義した変数空間を確認したいなら、以下でサンプリング結果を確認することができます'
('case 1', 5.2531236943393465)
('case 2', -1.1447309545664304)
('case 1', 1.671139091906336)
('case 2', -8.76062390252624)
('case 1', 1.113744767831003)
('case 2', 3.7503944297243876)
('case 2', 8.72919330433156)
('case 1', 1.7095182325330534)
('case 2', 1.3603761669055636)
('case 2', -4.894127500903174)
'--- 次に、探索を行います ---'
100%|██████████| 100/100 [00:00<00:00, 246.51it/s, best loss: 0.01602711992416122]
'=== best ==='
{'a': 1, 'c2': -0.12659826193183388}
'=== space_eval ==='
('case 2', -0.12659826193183388)
''


# 2.1パラメータ式

hyperoptの最適化アルゴリズムに用いられる変数スペースの定義は、以下のようなものがあります。

## hp.choice(label, options)

オプションの1つを返します。リストまたはタプルでなければなりません。 optionsの要素は、それ自体が確率的な式で [入れ子] にすることができます。この場合、いくつかのオプションにしか現れない確率的選択が 条件 パラメータになる。


## hp.randint(label, upper)

[0, upper] の範囲の乱数を返します。この分布のセマンティクスは、より遠い整数値と比較して、近くの整数値間の損失関数に相関がないことです。これは、例えばランダムシードを記述するための適切な分布です。損失関数がおそらく近くの整数値に相関する場合は、quniform 、 qloguniform 、 qnormal または qlognormal のいずれかのような「量子化」連続分布の1つを使用するべきです。


## hp.uniform(label, low, high)

low と high の間で均等に値を返します。
最適化するとき、この変数は両側の間隔に制約されます。


## hp.quniform(label, low, high, q)

round(uniform(low, high) / q) * q のような値を返します。
目的が幾分「滑らか」であるが、上と下の両方に拘束されるべき離散値に適している。


## hp.loguniform(label, low, high)

exp(uniform(low, high)) のような対数に一様に分布するように返します。
最適化すると、この変数は区間 [exp(low), exp(high)] に制約されます。


## hp.qloguniform(label, low, high, q)

round(exp(uniform(low, high)) / q) * q のような値を返します。
目的が「滑らか」で、値の大きさがより滑らかになるが、上と下の両方に限定される離散変数に適しています。


## hp.normal(label, mu, sigma)

平均ミューと標準偏差 sigma で正規分布している実際の値を返します。最適化する場合、これは制約のない変数です。


## hp.qnormal(label, mu, sigma, q)

round(normal(mu, sigma) / q) * q のような値を返します。
おそらく mu 付近の値をとるが、根本的に無制限の離散変数に適しています。


## hp.lognormal(label, mu, sigma)

戻り値の対数が正規分布するように exp(normal(mu, sigma)) に従って描画された値を返します。最適化すると、この変数は正の値に制限されます。


## hp.qlognormal(label, mu, sigma, q)

round(exp(normal(mu, sigma)) / q) * q のような値を返します。
目的が滑らかで、一方の側から境界を定められている変数のサイズによってスムーズになる離散変数に適しています。

# 次に sklearn と組み合わせて hyper parameter の調整を行う。

参考にしたURLは、　https://blog.amedama.jp/entry/hyperopt　です。

余談ですが、英語で検索するよりも日本語で検索したほうが、良い記事が多いのはなぜだろうか。

母国語である日本語での検索能力が高いからなのか、日本語の記事が優秀だからだろうか。


In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from functools import partial

from hyperopt import hp
from hyperopt import fmin
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import space_eval

from sklearn import datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.svm import SVC


def objective(X, y, args):
    """最小化したい目的関数"""
    # モデルのアルゴリズムに SVM を使う
    model = SVC(**args)
    # Stratified 5 Fold Cross Validation
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_validate(model, X=X, y=y, cv=kf)
    # 最小化なので符号を反転する
    return -1 * scores['test_score'].mean()


def main():
    # データセットを読み込む
    dataset = datasets.load_iris()
    X, y = dataset.data, dataset.target
    # 目的関数にデータセットを部分適用する
    f = partial(objective, X, y)
    # 変数の値域を定義する
    space = {
        # 底が自然対数なので 2.303 をかけて常用対数に変換する
        'C': hp.loguniform('C', 2.303 * 0, 2.303 * +2),
        'gamma': hp.loguniform('gamma', 2.303 * -2, 2.303 * +1),
        'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
    }
    # 探索過程を記録するオブジェクト
    trials = Trials()
    # 目的関数を最小化するパラメータを探索する
    best = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=100, trials=trials)
    
    print('--- 結果を出力する ---')
    print(space_eval(space, best))

if __name__ == '__main__':
    main()


100%|██████████| 100/100 [00:03<00:00, 25.86it/s, best loss: -0.9933333333333334]
'--- 結果を出力する ---'
{'C': 3.7854270724091728, 'gamma': 0.061837754861085684, 'kernel': 'rbf'}


# 次に最近、論文でも見かけるようになった、Train, Validation, Test の3分割データセットをつかったCrossValidationを実装してみます。

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from functools import partial

from hyperopt import hp
from hyperopt import fmin
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import space_eval

from sklearn import datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.svm import SVC

import random
import numpy as np


def split_train_validation_test(X, y, tra_val_tes_rate=[0.6, 0.2, 0.2]):
    # train, validation, test に三分割する
    random_index = list(range(len(y)))
    random.shuffle(random_index)
    tvt_sep = [int(rate*len(y)) for rate in tra_val_tes_rate]
    tvt_sep = [sum(tvt_sep[:i+1]) for i in range(len(tvt_sep))]
    tvt_index = random_index[0:tvt_sep[0]], random_index[tvt_sep[0]:tvt_sep[1]], random_index[tvt_sep[1]:tvt_sep[2]]
    train_X, train_y = X[tvt_index[0]], y[tvt_index[0]]
    validation_X, validation_y = X[tvt_index[1]], y[tvt_index[1]]
    test_X, test_y  = X[tvt_index[2]], y[tvt_index[2]]
    return train_X, train_y, validation_X, validation_y, test_X, test_y

def objective(train_X, train_y, val_X, val_y, args):
    """最小化したい目的関数"""
    # モデルのアルゴリズムに SVM を使う
    model = SVC(**args)
    model.fit(train_X, train_y)
    predict = model.predict(val_X)
    errors = predict - val_y    
    # 最小化なので符号を反転する
    return np.abs(errors).mean()


# データセットを読み込む
dataset = datasets.load_iris()
X, y = dataset.data, dataset.target

train_X, train_y, validation_X, validation_y, test_X, test_y = split_train_validation_test(X, y)

# 目的関数にデータセットを適応する
f = partial(objective, train_X, train_y, validation_X, validation_y)
# 変数の値域を定義する
space = {
    # 底が自然対数なので 2.303 をかけて常用対数に変換する
    'C': hp.loguniform('C', 2.303 * 0, 2.303 * +2),
    'gamma': hp.loguniform('gamma', 2.303 * -2, 2.303 * +1),
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
}
# 探索過程を記録するオブジェクト
trials = Trials()
# 目的関数を最小化するパラメータを探索する
best = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print('--- 最適なハイパーパラメータを確認する ---')
print(space_eval(space, best))
print('--- テストスコアを計算する ---')
model = SVC(**space_eval(space, best))
model.fit(train_X, train_y)
predict = model.predict(test_X)
errors = predict - test_y    
print(np.abs(errors).mean())

# 変数探索に使われるalgoの種類について


公式ドキュメント （https://hyperopt.github.io/hyperopt/）　によれば、現在(2019/3/30)実装されている探索アルゴリズムは下記の２つのみ。

1. Random Search
2. Tree of Parzen Estimators (TPE)

Gaussian processesやregression treeをに基づいた混合ベイズ最適化モデル？（accommodate Bayesian optimization algorithms）というものがある。これに基づいた実装としてhyperopt は開発されたが、まだ実装されてはいない。

Random Search についてはここでは言及しない。なぜなら、簡単だと思うから。

TPE　について言及します。


## パラメータの自動調整技術の変遷

[NeuPy] によると、パラメータの自動調整の技術の変遷は、SMBO　-> gaussian process -> TPE となっているそうです。

TPEは2011年にNIPSで発表された論文(https://www.lri.fr/~kegl/research/PDFs/BeBaBeKe11.pdf)です。

[NeuPy] http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html


## NeuPy の記事を読んでみよう。

NeuPy は Neural Networks のパラメータ最適化モジュールです。自動調整の技術の変遷と概要をまとめた記事があったので、簡単に和訳していきます。

詳細については、上記の[NeuPy]のURLを参照してください。


### 以下、和訳で要約

パラメータの自動調整の種類を、以下に列挙します。

1. Grid Search
2. Random Search
3. Hund-tuning
4. Gaussian Process with Expected Improvement
5. Tree-Structured Parzen Estimators (TPE)

ここでは、4.　と 5. について該当する記事を和訳しておきます。


#### Gaussian Process with Expected Improvement

(記事の場所：http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#bayesian-optimization)


* Bayesian Optimization

Bayesian Optimization　は　派生のある？（derivative-free） の最適化法です。いくつかの最適化タイプがありますが、ここでは　Acquisition Function を使った Gaussian Process に注目します。前述で紹介した　Hand-tuning での説明と似ていると思われるかもしれません。　Gaussian Process は前回のパラメータと精度の結果を使い、次の未観測パラメータを仮説します。Axquisition Function はこれらの情報を使い、次のパラメータセットを提案します。

* Gaussian Process

(内田コメント)　Gaussian Process の一般的理解がないと、以下を読み解くのは難しかった。これ以降でもsklearnのGaussian Processモジュールを応用した実装を行うので、下記をあらかじめ読んでおくべき。

[sklearnの解説記事]　

https://scikit-learn.org/stable/modules/gaussian_process.html

[sklearnの実装レファレンス] 

https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor

[内田ではよく分からなかったけど、数理的に説明している日本語記事] 

https://qiita.com/kilometer/items/6ab1d7f2cffc0c6e4a95

[頭のいい人が「分かりやすい」と言っていたスライド資料。その１]　

https://www.ism.ac.jp/~daichi/lectures/H26-GaussianProcess/gp-lecture1-matsui.pdf

[頭のいい人が「分かりやすい」と言っていたスライド資料。その２]　　←　オススメ！

https://www.ism.ac.jp/~daichi/lectures/H26-GaussianProcess/gp-lecture2-daichi.pdf



Gaussian Process の背景には、インプット x は　アウトプット　y　= f(x) を持ち、f は確率関数である、という考えがあります。この確率関数はアウトプットをgaussian分布からサンプリングします。また、インプット　x はgaussian分布に帰属しているといえます。つまり、インプットxを生成されるgaussian　proccessは、いくつかのgaussian分布の平均μと標準偏差σを定義します。

Gaussian Process は　Multivariate Gaussian Distribution(MGD) に基づいた生成です。MGDは平均ベクトルと共分散ベクトルで定義されます。一方で、Gaussian Process は平均関数(mean function)と共分散関数（covariance function）で定義されます。基本的には、関数は無限ベクトルです（取り得る値が無限で、ベクトルで返却する関数ということ？）。また、多変量ガウス分布は離散的な数の可能な入力を持つ関数に対するガウス過程であると言えます。

例えば、二次元のパラメータベクトル　x　を多変量ガウス分布で生成する場合、以下のようなサンプリングとなります。

![MGDE01](img/MGDE01.png)

これを違う可視化にすると、下記のようになります。結ばれた線は同時にサンプリングされたことを示しています。このグラフであれば、パラメータベクトルの次元を増やしても表記することができます。

![MGDE02](img/MGDE02.png)

パラメータベクトルが5次元で、十分な回数サンプリングすれば下記のようになります。

![MGDE03](img/MGDE03.png)

つまり、最適なパラメータベクトルをサンプリングするためには、この多変量ガウス分布の平均ベクトルと共分散行列をなんらかの学習過程で推定し、収束させれば良いのです。

例えば、３層のニューラルネットワークの中間hidden層のニューロン数をパラメータであるとしましょう。
ニューロン数を2回サンプリングして、それぞれの精度を観測します。
すると下図のような、パラメータ（X軸）と精度（Y軸）の予測が更新されます。
後は、iterationを重ねて、サンプリングすべき範囲を狭めていきます。
つまり、下記のようになります。

![MGDE04](img/MGDE04.png)

しかし、前述のGausian Processでは分散を設定しているので、上図のように観測点が一点にはなりません。その精度を決める予測値は決定関数から得られたものであり、その決定関数はほとんどのneural networkにとって正しくないのです(注釈1)。観測されるノイズを決定しているGausian Processの分散パラメータを変更することで、これを修正する。この工夫はそれほど確かではない予測だけでなく、観測データ点を通過しない隠れユニット数の平均を与えます。


[注釈1] この意味が分かる人がいればフォローをお願いします。内田の解釈では、パラメータが決まった状態で学習した結果を、決定関数（a deterministic function）と表現しているのだと思います。決定関数がテストセットの変数から予測値を出力します。そして、予測値とテストセットの正解値から誤差を計算して精度指標（Accuracy）が計算されます。よって、精度指標はパラメータが固定されていても、テストセットと学習結果の重みが確率的に変動すれば分散します。テストセットは一般的に乱数抽出されますし、重みも学習過程に乱数要素を含みます。よって、観測点においても精度は一意に決まらず分散します。

![MGDE05](img/MGDE05.png)


* Acquisition Function

Acquisition Function は次のステップのパラメータセットを決定します。その種類は複数あります。その内の代表的なものが、Expected Improvement です。もし、最小化を目的として次のパラメータセットを探索する場合は、次の数式を用います。

\begin{align}
g_{min}(x)  = max(0, y_{min} - y_{lowest\ expected}) \\ 
\end{align}


\begin{align}
y_{min} : 観測されたyの最小値
\end{align}
\begin{align}
y_{lowest\ expected} : 各xに関連づけられた信頼区間内の最小値。
\end{align}

今回の隠れ層のユニット数を探索する場合は、Accuracyの最大化問題なので、以下を用います。

\begin{align}
g_{max}(x)  = max(0, y_{highest\ expected} - y_{max}) \\ 
\end{align}

このExpected Improvement function に従えば、各ステップの次のxは下記のように選出されます。

![MGDE06](img/MGDE06.png)


* Find number of hidden units

それでは、Gaussian Process regression　と　Expected Improvement　function に基づいたハイパーパラメータの探索を実装してみます。ひきつづき、最適な隠れユニット数を探索する問題にとりくみます。おなじみの手書き数値の判別問題です。

まずは、学習、ニューラルネットワーク、予測誤差を関数で定義しましょう。

In [ ]:
# numpy への依存部分にエラーがあり、importできなかった。
# とはいえ、コードに目を通すだけでプロセスを理解することはできるので問題ないはず。
from neupy import algorithms, layers 

################################
# ニューラルネットワークの定義とその学習。戻り値はテストセットの精度スコアになっている。
def train_network(n_hidden, x_train, x_test, y_train, y_test):
    network = algorithms.Momentum(
        [
            layers.Input(64),
            layers.Relu(n_hidden),
            layers.Softmax(10),
        ],

        # Randomly shuffle dataset before each
        # training epoch.
        shuffle_data=True,

        # Do not show training progress in output
        verbose=False,

        step=0.001,
        batch_size=128,
        loss='categorical_crossentropy',
    )
    network.train(x_train, y_train, epochs=100)

    # Calculates categorical cross-entropy error between
    # predicted value for x_test and y_test value
    return network.score(x_test, y_test)


################################
# 次に手書きデータセットをscikit-learnで読み込む。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from neupy import utils

utils.reproducible()

dataset = datasets.load_digits()
n_samples = dataset.target.size
n_classes = 10

# One-hot encoder
target = np.zeros((n_samples, n_classes))
target[np.arange(n_samples), dataset.target] = 1

x_train, x_test, y_train, y_test = train_test_split(
    dataset.data, target, test_size=0.3
)


################################
# パラメータの選択プロセスを定義する。
# はじめに、Gaussian Process regression を実行し特定のベクトル入力に対して予測の平均と標準偏差を返却する関数を定義します。

import numpy as np
from sklearn.gaussian_process import GaussianProcess

def vector_2d(array):
    return np.array(array).reshape((-1, 1))

def gaussian_process(x_train, y_train, x_test):
    '''
    上記の引数名だと、まるで手書きデータセットを受け取ると誤解してしまう。
    実際に引数にとるのは、下記のデータである。
    
    x_train = 検証済みのパラメータが各ステップごとに格納されている 
    y_train = 検証結果の精度スコアが各ステップごとに格納されている
    x_test　= 次のステップで選択されるパラメータの候補が格納されている
    '''
    x_train = vector_2d(x_train)
    y_train = vector_2d(y_train)
    x_test = vector_2d(x_test)

    # Train gaussian process
    gp = GaussianProcess(corr='squared_exponential',
                         theta0=1e-1, thetaL=1e-3, thetaU=1)
    gp.fit(x_train, y_train)

    # Get mean and standard deviation for each possible
    # number of hidden units
    # 取り得る隠れユニット数ごとの平均と標準偏差を取得します。
    y_mean, y_var = gp.predict(x_test, eval_MSE=True)
    y_std = np.sqrt(vector_2d(y_var))

    return y_mean, y_std


####################################
#  予測結果を　Expected Improvement(EI) に適応し、次の最適ステップを探索します。

def next_parameter_by_ei(y_min, y_mean, y_std, x_choices):
    # Calculate expecte improvement from 95% confidence interval
    expected_improvement = y_min - (y_mean - 1.96 * y_std)
    expected_improvement[expected_improvement < 0] = 0

    max_index = expected_improvement.argmax()
    # Select next choice
    next_parameter = x_choices[max_index]

    return next_parameter


####################################
# 最後に、上記で定義したプロシージャを一つの関数にまとめます。

import random

def hyperparam_selection(func, n_hidden_range, func_args=None, n_iter=20):
    ''' 使用例
    best_n_hidden = hyperparam_selection(
        train_network,　# 前述で定義した関数
        n_hidden_range=[50, 1000],
        func_args=[x_train, x_test, y_train, y_test],
        n_iter=6,)
    '''
    if func_args is None:
        func_args = []

    scores = []
    parameters = []

    min_n_hidden, max_n_hidden = n_hidden_range
    n_hidden_choices = np.arange(min_n_hidden, max_n_hidden + 1)

    # To be able to perform gaussian process we need to
    # have at least 2 samples.
    # gaussian processをとりあえず稼働するためのn_hiddenを乱数抽出します。
    n_hidden = random.randint(min_n_hidden, max_n_hidden)
    score = func(n_hidden, *func_args)
    
    # parameters に検証したn_hiddenを追加。そして、その精度スコアをscoresに追加。
    parameters.append(n_hidden)
    scores.append(score)

    n_hidden = random.randint(min_n_hidden, max_n_hidden)

    for iteration in range(2, n_iter + 1):
        score = func(n_hidden, *func_args)

        parameters.append(n_hidden)
        scores.append(score)
        
        # 検証済みのparameters と score に、gaussian_processを実行して次のステップで検証すべきn_hiddenを選ぶ。
        y_min = min(scores)
        y_mean, y_std = gaussian_process(parameters, scores,
                                         n_hidden_choices)

        n_hidden = next_parameter_by_ei(y_min, y_mean, y_std,
                                        n_hidden_choices)

        if y_min == 0 or n_hidden in parameters:
            # Lowest expected improvement value have been achieved
            break

    # 結果、精度スコアが最小値となったパラメータを返却する
    min_score_index = np.argmin(scores)
    return parameters[min_score_index]


上記のプログラムで定義された手続きのイメージは、以下のようなものである。

![MGDE07](img/MGDE07.png)